In [1]:
import pandas as pd

In [2]:
from enum import Enum
class SongId(Enum):
    Keaton  = 1
    Alyse   = 2
    Morty   = 3
    Jay     = 4
    Richard = 5   

In [3]:
class Sensor(Enum):
    AF3 = 0
    F7 = 1
    F3 = 2
    FC5 = 3
    T7 = 4
    P7 = 5
    O1 = 6
    O2 = 7
    P8 = 8
    T8 = 9
    FC6 = 10
    F4 = 11
    F8 = 12
    AF4 = 14

Set the global settings

In [4]:
alpha_dir         = "../tf_alpha/"
beta_dir          = "../tf_beta/"
user              = "Richard"
points_per_sample = 100

alpha_sensors = ["F4",  "F7",  "F8"]
beta_sensors  = ["AF3", "AF4", "FC5", "F4"]

Read in files

In [5]:
import os
alpha_files = os.listdir(alpha_dir)
beta_files  = os.listdir(beta_dir)
print("Alpha Files Found: %d" % len(alpha_files))
print("Beta Files Found: %d" % len(beta_files))

Alpha Files Found: 100
Beta Files Found: 100


Remove all files that aren't a reading of the users song

In [6]:
def reading_of_user_song(filename):
    song_id = filename.split('_')[1]
    
    if song_id == 's':
        return False
    
    return int(song_id) == SongId[user].value

In [7]:
def remove_all_songs_from_other_users(file_list):
    i = 0
    while i < len(file_list):
        if reading_of_user_song(file_list[i]):
            i += 1
        else:
            file_list.pop(i)

In [8]:
remove_all_songs_from_other_users(alpha_files)
remove_all_songs_from_other_users(beta_files)
print("Alpha files with user song found: %d" % len(alpha_files))
print("Beta file with user song found: %d" % len(beta_files))

Alpha files with user song found: 16
Beta file with user song found: 16


Split the files into authorized and attack sets

Authorized: The user is listening to their chosen song
Attack:     Someone else listening to the user's song

In [9]:
def is_auth(filename):
    return filename.split('_')[0] == user

In [10]:
alpha_auth = []
beta_auth  = []
alpha_att  = []
beta_att   = []

for f in alpha_files:
    #print("{}: {}".format(f, is_auth(f)))
    if is_auth(f):
        alpha_auth.append(f)
    else:
        alpha_att.append(f)
        
for f in beta_files:
    #print("{}: {}".format(f, is_auth(f)))
    if is_auth(f):
        beta_auth.append(f)
    else:
        beta_att.append(f)
        
# print(alpha_auth)
# print(alpha_att)
# print(beta_auth)
# print(beta_att)

del alpha_files
del beta_files

Beginning reading in data

In [11]:
import statistics as stats
def get_avgs(l, n):
    """
    return a list of averages using each set of n
    elements in order from n
    """
    
    i = 0
    avgs = []
    while i + n < len(l):
        avgs.append(stats.mean(l[i:i+n]))
        i += n
    
    return avgs

Create containers for data

In [12]:
alpha_tag = "_ALPHA"
beta_tag  = "_BETA"

auth_data = dict()
att_data  = dict()

for sensor in alpha_sensors:
    auth_data[sensor + alpha_tag] = []
    att_data[sensor + alpha_tag]  = []
    
for sensor in beta_sensors:
    auth_data[sensor + beta_tag] = []
    att_data[sensor + beta_tag]  = []

Read in and process data from authorized user files

In [13]:
for f in alpha_auth:
    df = pd.read_csv(alpha_dir + f)
    for sensor in alpha_sensors:
        key = sensor + alpha_tag
        auth_data[key] += get_avgs(df[key].values, points_per_sample)        
        
for f in beta_auth:
    df = pd.read_csv(beta_dir + f)
    for sensor in beta_sensors:
        key = sensor + beta_tag
        auth_data[key] += get_avgs(df[key].values, points_per_sample)

In [14]:
for key in auth_data.keys():
    print("{}: {}".format(key, len(auth_data[key])))

F4_ALPHA: 64
F7_ALPHA: 64
F8_ALPHA: 64
AF3_BETA: 64
AF4_BETA: 64
FC5_BETA: 64
F4_BETA: 64


Read in and process data from attacker files

In [15]:
for f in alpha_att:
    df = pd.read_csv(alpha_dir + f)
    for sensor in alpha_sensors:
        key = sensor + alpha_tag
        att_data[key] += get_avgs(df[key].values, points_per_sample)        
        
for f in beta_att:
    df = pd.read_csv(beta_dir + f)
    for sensor in beta_sensors:
        key = sensor + beta_tag
        att_data[key] += get_avgs(df[key].values, points_per_sample)

In [16]:
for key in auth_data.keys():
    print("{}: {}".format(key, len(att_data[key])))

F4_ALPHA: 192
F7_ALPHA: 192
F8_ALPHA: 192
AF3_BETA: 192
AF4_BETA: 192
FC5_BETA: 192
F4_BETA: 192


In [17]:
Print to File

SyntaxError: invalid syntax (<ipython-input-17-c5bf4e0ef332>, line 1)

In [ ]:
auth_df = pd.DataFrame(auth_data)
auth_df.to_csv("auth.csv")

att_df  = pd.DataFrame(att_data)
att_df.to_csv("att.csv")